# Understanding the Data: From the file `heart-disease.names`

data_path = 'data/heart+disease/processed.cleveland.data'

From the file `heart-disease.names`, the data has 14 attributes and 303 instances. After close examination, of the 14 attributes, 13 are used for prediction and the 14th attribute is the predicted attribute. Of all the attributes, 9 are categorical and 5 are continuous.

### Categorical Attributes
- sex: 1 = male; 0 = female
- cp: chest pain type
    - Value 1: typical angina
    - Value 2: atypical angina
    - Value 3: non-anginal pain
    - Value 4: asymptomatic
- fbs: fasting blood sugar > 120 mg/dl (1 = true; 0 = false)
- restecg: resting electrocardiographic results
    - Value 0: normal
    - Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
    - Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria
- exang: exercise induced angina (1 = yes; 0 = no)
- slope: the slope of the peak exercise ST segment
    - Value 1: upsloping
    - Value 2: flat
    - Value 3: downsloping
- ca: number of major vessels (0-3) colored by flourosopy
- thal: 3 = normal; 6 = fixed defect; 7 = reversible defect
- num: diagnosis of heart disease (angiographic disease status)

### The Dataset

I'll be using the `processed.cleveland.data` dataset for this analysis.

